In [ ]:
!pip install gensim
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
data=pd.read_json('/content/Magazine_Subscriptions.jsonl.gz',lines=True)

In [ ]:
data.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,Great pictures and recipes,Wonderful recipes in this magazine.,[],B00HLSSQKK,B00HLSSQKK,AE7Y5RLYIKHOZB5NKKOEKYG2SPSQ,2020-12-15 15:29:02.433,0,True
1,4,great for kids who love sports!,Great sports magazine that's on my 9 year olds...,[],B002PXW04Y,B002PXW04Y,AHORTSSMI6ZZFUFWMPT4UFR2ISUQ,2016-08-12 03:28:32.000,1,True
2,5,A great look at what's new on the kosher scene...,"""Joy of Kosher"" magazine fills a much-needed n...",[],B00HCR5090,B00HCR5090,AFW2PDT3AMT4X3PYQG7FJZH5FXFA,2014-04-06 23:32:53.000,4,False
3,5,"If you enjoy reading an eye-catching, easy-to-...",I've been addicted to Martha Stewart's Everyda...,[],B0000ARXXS,B0000ARXXS,AFW2PDT3AMT4X3PYQG7FJZH5FXFA,2007-06-15 04:04:15.000,4,False
4,1,Too many “ gear ads” !!,Too many ads!,[],B0025ZOVEO,B0025ZOVEO,AFUB7CHTXRPD447QVQCHBZVN2IPQ,2020-06-01 21:24:40.246,13,True


In [ ]:
data.text[0]

'Wonderful recipes in this magazine.'

In [ ]:
import gensim
data.text = data.text.apply(gensim.utils.simple_preprocess)

In [ ]:
data.text[0]

['wonderful', 'recipes', 'in', 'this', 'magazine']

In [ ]:
data.shape

(71497, 10)

In [ ]:
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Function to remove stopwords
def remove_stopwords(tokens):
    # Convert list of tokens into a spaCy Doc
    doc = nlp(" ".join(tokens))
    # Filter out stopwords and return the remaining tokens as a list
    return [token.text for token in doc if not token.is_stop]

# Function to lemmatize tokens
def lemmatize_tokens(tokens):
    # Convert list of tokens into a spaCy Doc
    doc = nlp(" ".join(tokens))
    # Extract lemmatized tokens and return as a list
    return [token.lemma_ for token in doc if not token.is_stop]

# Apply the stopword removal function to the 'text' column
data['text_no_stopwords'] = data.text.apply(remove_stopwords)

# Apply lemmatization to the 'text_no_stopwords' column
data['text_lemmatized'] = data.text_no_stopwords.apply(lemmatize_tokens)

# Display the results
print(data[['text', 'text_no_stopwords', 'text_lemmatized']].head())

                                                text  \
0           [wonderful, recipes, in, this, magazine]   
1  [great, sports, magazine, that, on, my, year, ...   
2  [joy, of, kosher, magazine, fills, much, neede...   
3  [ve, been, addicted, to, martha, stewart, ever...   
4                                   [too, many, ads]   

                                   text_no_stopwords  \
0                     [wonderful, recipes, magazine]   
1       [great, sports, magazine, year, olds, level]   
2  [joy, kosher, magazine, fills, needed, niche, ...   
3  [ve, addicted, martha, stewart, everyday, food...   
4                                              [ads]   

                                     text_lemmatized  
0                      [wonderful, recipe, magazine]  
1         [great, sport, magazine, year, old, level]  
2  [joy, kosher, magazine, fill, need, niche, kos...  
3  [ve, addict, martha, stewart, everyday, food, ...  
4                                               [ad

In [ ]:
from gensim.models import Word2Vec

# Train Word2Vec on lemmatized text
word2vec_model = Word2Vec(
    window=5,
    min_count=2,
    workers=4
)

# Save Word2Vec model for future use
word2vec_model.save("word2vec_model.model")

In [ ]:
word2vec_model.build_vocab(data.text_lemmatized)
word2vec_model.train(data.text_lemmatized,total_examples=word2vec_model.corpus_count,epochs=word2vec_model.epochs)

(5157235, 6237490)

In [ ]:
import numpy as np

# Function to get the average word vector for a text
def get_avg_word2vec(tokens, model, vector_size):
    # Filter tokens that exist in the Word2Vec vocabulary
    valid_tokens = [token for token in tokens if token in model.wv]
    if len(valid_tokens) == 0:
        return np.zeros(vector_size)  # Return a zero vector if no valid tokens
    return np.mean(model.wv[valid_tokens], axis=0)

# Apply to the dataset
data['text_vector'] = data.text_lemmatized.apply(
    lambda x: get_avg_word2vec(x, word2vec_model, vector_size=100)
)

In [ ]:
data['rating'].value_counts()

,count
rating,
5,44620
1,9685
4,8206
3,5033
2,3953


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
# Features (X) and target (y)
X = np.stack(data['text_vector'].values)  # Stack vectors into a matrix
y = data['rating']  # Assuming 'ratings' is your target column

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Check the new distribution of the training data
print(f"Original dataset shape: {y_train.value_counts()}")
print(f"Resampled dataset shape: {y_train_smote.value_counts()}")

Original dataset shape: rating
5    35696
1     7748
4     6565
3     4026
2     3162
Name: count, dtype: int64
Resampled dataset shape: rating
5    35696
2    35696
1    35696
3    35696
4    35696
Name: count, dtype: int64


In [ ]:
#LOGISTIC REGRESSION
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_smote, y_train_smote)

# Make predictions
y_pred = classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.5376223776223776
Classification Report:
               precision    recall  f1-score   support

           1       0.53      0.58      0.55      1937
           2       0.18      0.35      0.24       791
           3       0.20      0.30      0.24      1007
           4       0.19      0.35      0.25      1641
           5       0.89      0.61      0.72      8924

    accuracy                           0.54     14300
   macro avg       0.40      0.44      0.40     14300
weighted avg       0.67      0.54      0.58     14300



In [ ]:
#RANDOM FOREST
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42,criterion='entropy')
rf_classifier.fit(X_train_smote, y_train_smote)
y_pred_rf = rf_classifier.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.6352447552447552
Classification Report:
               precision    recall  f1-score   support

           1       0.53      0.66      0.59      1937
           2       0.22      0.22      0.22       791
           3       0.24      0.24      0.24      1007
           4       0.22      0.21      0.21      1641
           5       0.83      0.79      0.81      8924

    accuracy                           0.64     14300
   macro avg       0.41      0.42      0.41     14300
weighted avg       0.64      0.64      0.64     14300



In [ ]:
# Preprocess the new sentence
new_sentence = "The magazine is excellent and I enjoy reading it."  # Example input
tokens = gensim.utils.simple_preprocess(new_sentence)  # Tokenize
tokens_no_stopwords = remove_stopwords(tokens)  # Remove stopwords
tokens_lemmatized = lemmatize_tokens(tokens_no_stopwords)  # Lemmatize

# Convert the preprocessed sentence to a vector
new_vector = np.mean([word2vec_model.wv[word] for word in tokens_lemmatized if word in word2vec_model.wv], axis=0).reshape(1, -1)

# Predict the rating
predicted_rating = rf_classifier.predict(new_vector)
print(f"Predicted Rating: {predicted_rating[0]}")

Predicted Rating: 5


In [ ]:
# Preprocess the new sentence
new_sentence = "The magazine is rubbish and is too bias towards media.Never ever gonna read it again.Worst magazine"  # Example input
tokens = gensim.utils.simple_preprocess(new_sentence)  # Tokenize
tokens_no_stopwords = remove_stopwords(tokens)  # Remove stopwords
tokens_lemmatized = lemmatize_tokens(tokens_no_stopwords)  # Lemmatize

# Convert the preprocessed sentence to a vector
new_vector = np.mean([word2vec_model.wv[word] for word in tokens_lemmatized if word in word2vec_model.wv], axis=0).reshape(1, -1)

# Predict the rating
predicted_rating = rf_classifier.predict(new_vector)
print(f"Predicted Rating: {predicted_rating[0]}")

Predicted Rating: 1


In [ ]:
#DECISION TREE
dt=DecisionTreeClassifier(criterion='entropy',random_state=42)
dt.fit(X_train_smote, y_train_smote)
y_pred_dt=dt.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Classification Report:\n", classification_report(y_test, y_pred_dt))

Decision Tree Accuracy: 0.5013986013986014
Classification Report:
               precision    recall  f1-score   support

           1       0.45      0.45      0.45      1937
           2       0.15      0.24      0.19       791
           3       0.15      0.25      0.19      1007
           4       0.17      0.28      0.21      1641
           5       0.81      0.61      0.69      8924

    accuracy                           0.50     14300
   macro avg       0.34      0.36      0.34     14300
weighted avg       0.60      0.50      0.54     14300

